In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import frechet_video_distance as fvd
import numpy as np
# Number of videos must be divisible by 16.
NUMBER_OF_VIDEOS = 16
VIDEO_LENGTH = 15


In [2]:
vid1 = np.stack([np.load("./independent_10_7/sample_0000-0.npy"),
                 np.load("./independent_10_7/sample_0000-3.npy")])
vid2 = np.stack([np.load("./independent_10_7/sample_0001-0.npy"),
                 np.load("./independent_10_7/sample_0002-0.npy")])


# # hack to have number of videos be divisible by 16 
vid1 = np.repeat(vid1, 8, 0)
vid2 = np.repeat(vid2, 8, 0)

# reshape to have channel in last dim
vid1 = np.moveaxis(vid1, 2, 4) # B, T, H, W, C
vid2 = np.moveaxis(vid2, 2, 4) # B, T, H, W, C

In [5]:
vid1.shape

(16, 300, 64, 64, 3)

In [12]:
def compute_FVD_features(vid1, vid2): 
    # batch has to be == 16 in fvd.create_id3_embedding
    assert vid1.shape[0] == 16 
    assert vid2.shape[0] == 16

    # assert channel is last dimension
    # and trust that shape is B, T, H, W, C
    assert vid1.shape[4] == 3
    assert vid2.shape[4] == 3
    
    with tf.Graph().as_default():
        vid1 = tf.convert_to_tensor(vid1, np.uint8)
        vid2 = tf.convert_to_tensor(vid2, np.uint8)
        
        vid1_feature_vec = fvd.create_id3_embedding(fvd.preprocess(vid1,(224, 224)))
        vid2_feature_vec = fvd.create_id3_embedding(fvd.preprocess(vid2,(224, 224)))
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            return sess.run(vid1_feature_vec), sess.run(vid2_feature_vec)


In [13]:
def compute_FVD(vid1_features, vid2_features): 
    # no batch limitations here, can pass in entire dataset
    with tf.Graph().as_default():
        vid1_features = tf.convert_to_tensor(vid1_features, np.float32)
        vid2_features = tf.convert_to_tensor(vid2_features, np.float32)
        
        result = fvd.calculate_fvd(vid1_features, vid2_features)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            return sess.run(result)


In [15]:
%%time
vid1_feature_vec, vid2_feature_vec = compute_FVD_features(vid1[:,:15,:,:,:], vid2[:,:15,:,:,:])

2022-04-21 16:56:46.905012: W tensorflow/core/common_runtime/graph_constructor.cc:1511] Importing a graph with a lower producer version 26 into an existing graph with producer version 987. Shape inference will have run different parts of the graph with different producer versions.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2022-04-21 16:56:49.631918: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-21 16:56:53.861581: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-04-21 16:57:00.360584: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 308281344 exceeds 10% of free system memory.
2022-04-21 16:57:08.834691: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192675840 exceeds 10% of free system memory.
2022-04-21 16:57:14.900173: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memor

CPU times: user 1min 21s, sys: 2.45 s, total: 1min 23s
Wall time: 47.7 s


In [18]:
vid1_feature_vec.shape

(16, 400)

In [19]:
result = compute_FVD(vid1_feature_vec, vid2_feature_vec)
result

1055.9459